<h1><font color="#113D68" size=6>Deep Learning con Python y Keras</font></h1>

<h1><font color="#113D68" size=5>Parte 6. Redes Neuronales Recurrentes</font></h1>

<h1><font color="#113D68" size=4>6. Práctica: Procesamiento del Lenguaje Natural</font></h1>

<br><br>
<div style="text-align: right">
<font color="#113D68" size=3>Manuel Castillo Cara</font><br>

</div>

---

<a id="indice"></a>
<h2><font color="#004D7F" size=5>Licencia</font></h2>

<p><small><small>Copyright 2022 Manuel Castillo Cara.</p>
<p><small><small> Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at </p>
<p><small><small> <a href="https://www.apache.org/licenses/LICENSE-2.0">https://www.apache.org/licenses/LICENSE-2.0</a> </p>
<p><small><small> Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License. </p>
    
---

---

<a id="indice"></a>
<h2><font color="#004D7F" size=5>Índice</font></h2>

* [0. Contexto](#section0)
* [1. Descripción del problema: generación de texto](#section1)
* [2. LSTM de linea base](#section2)
    * [2.1. Cargar el dataset](#section2.1)
    * [2.2. Conversión a numérico](#section2.2)
    * [2.3. Dimensiones del dataset](#section2.3)
    * [2.4. Procesamiento de datos](#section2.4)
    * [2.5. Diseño de la LSTM](#section2.5)
    * [2.6. Crear puntos de control](#section2.6)
    * [2.7. Resultados](#section2.7)
* [3. Generación de texto con LSTM](#section3)
    * [3.1. Cargar los pesos de LSTM](#section3.1)
    * [3.2. Convertir de entero a carácter](#section3.2)
    * [3.3. Resultados y evaluación](#section3.3)
* [4. LSTM más profunda](#section4)
* [5. Mejorar nuestro modelo](#section5)

---
<a id="section0"></a>
# <font color="#004D7F" size=6> 0. Contexto</font>

En este proyecto, aprenderemos a crear un modelo generativo de texto utilizando LSTM:
* Descripción de un modelos generativos de texto.
* Enmarcar el problema de las secuencias de texto a un modelo generativo.
* Desarrollar una LSTM para generar secuencias de texto.

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section1"></a>
# <font color="#004D7F" size=6>1. Descripción del problema: generación de texto</font>

VAmos a trabajar el libro de "Alicia en el país de las maravillas", por lo que podemos descargarlo de la página (Texto sin formato UTF-8) de este libro de forma gratuita y colocarlo en su directorio de trabajo con el nombre `wonderland.txt`. 

Abrimos el archivo y eliminamos el encabezado:
```
        *** START OF THIS PROJECT GUTENBERG EBOOK ALICE'S ADVENTURES IN WONDERLAND ***
```

El pie de página es todo el texto después de la línea de texto que dice:
```
        THE END
```

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i>
Más información sobre el [Proyecto Gutenberg](https://www.gutenberg.org/)

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i>
Descargar el libro [Alice's Adventures in Wonderland](https://www.gutenberg.org/ebooks/11)

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section2"></a>
# <font color="#004D7F" size=6>2. LSTM de linea base</font>

En esta sección desarrollaremos una red LSTM simple para aprender secuencias de caracteres de Alicia en el país de las maravillas. 

<a id="section2.1"></a>
# <font color="#004D7F" size=5>2.1. Cargar el dataset</font>

Comencemos importando las clases y funciones que pretendemos usar para entrenar nuestro modelo.

Debemos cargar el texto ASCII y convertir todos los caracteres a minúsculas.

In [17]:
# Small LSTM Network to Generate Text for Alice in Wonderland
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

# load ascii text and covert to lowercase
filename = "data/wonderland.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()

<a id="section2.2"></a>
# <font color="#004D7F" size=5>2.2. Conversión a numérico</font>

No podemos modelar los caracteres directamente, sino que debemos convertir los caracteres a números enteros. 

Por ejemplo, la lista de caracteres en minúscula ordenados únicos en el libro es la siguiente:

In [18]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
char_to_int

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 '#': 4,
 '$': 5,
 '%': 6,
 "'": 7,
 '(': 8,
 ')': 9,
 '*': 10,
 ',': 11,
 '-': 12,
 '.': 13,
 '/': 14,
 '0': 15,
 '1': 16,
 '2': 17,
 '3': 18,
 '4': 19,
 '5': 20,
 '6': 21,
 '7': 22,
 '8': 23,
 '9': 24,
 ':': 25,
 ';': 26,
 '?': 27,
 '@': 28,
 '[': 29,
 ']': 30,
 '_': 31,
 'a': 32,
 'b': 33,
 'c': 34,
 'd': 35,
 'e': 36,
 'f': 37,
 'g': 38,
 'h': 39,
 'i': 40,
 'j': 41,
 'k': 42,
 'l': 43,
 'm': 44,
 'n': 45,
 'o': 46,
 'p': 47,
 'q': 48,
 'r': 49,
 's': 50,
 't': 51,
 'u': 52,
 'v': 53,
 'w': 54,
 'x': 55,
 'y': 56,
 'z': 57,
 'ù': 58,
 '—': 59,
 '‘': 60,
 '’': 61,
 '“': 62,
 '”': 63,
 '\ufeff': 64}

<a id="section2.3"></a>
# <font color="#004D7F" size=5>2.3. Dimensiones del dataset</font>

Ahora que se cargó el libro y se preparó el mapeo, podemos resumir el conjunto de datos.

In [19]:
# summarize the loaded data
# summarize the loaded data
n_chars = ???
n_vocab = ???
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  164202
Total Vocab:  65


En este tutorial, dividiremos el texto del libro en subsecuencias con una longitud fija de 100 caracteres, una longitud arbitraria. 

Cada patrón de entrenamiento de la red se compone de 100 pasos de tiempo de un carácter (X) seguidos de una salida de carácter (y). 

Por ejemplo, si la longitud de la secuencia es 5 (para simplificar), los dos primeros patrones de entrenamiento serían los siguientes:
```
            CHAPT -> E
            HAPTE -> R
```
A medida que dividimos el libro en estas secuencias, convertimos los caracteres a números enteros usando nuestra tabla de búsqueda que preparamos anteriormente.

In [20]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)


Total Patterns:  164102


Podemos ver que se han dividio en secuencias de 100

In [1]:
# Secuencia 0
dataX[0]
# Total Secuencia 0
len(dataX[0])
# Para esa secuencia de X le correspondería la salida
dataY[0]

NameError: name 'dataX' is not defined

<a id="section2.4"></a>
# <font color="#004D7F" size=5>2.4. Procesamiento de datos</font>

Ahora que hemos preparado nuestros datos de entrenamiento, necesitamos transformarlos. 
1. Transformar la lista de secuencias de entrada en la forma ***[muestras, pasos de tiempo, características].***
2. Cambiar la escala de los números enteros al rango de 0 a 1 (normalización) y usar la función de activación sigmoidea.
3. Convertir los patrones de salida (caracteres individuales convertidos en enteros) con One-Hot Encoding. 

In [21]:
# reshape X to be [samples, time steps, features]
??? 
print(X[0])
print(X[0][0])
print(X[0][0[0]])
# normalize
??? 

# one hot encode the output variable
??? 


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

<a id="section2.5"></a>
# <font color="#004D7F" size=5>2.5. Diseño de la LSTM</font>

Ahora podemos definir nuestro modelo LSTM. 
1. Definimos una única capa LSTM oculta con 256 unidades de memoria. 
2. La red utiliza un Dropout del 20%. 
3. La capa de salida es una capa densa que utiliza la función de activación Softmax. 
4. Compilación con pérdida logarítmica (`categorical_crossentropy`)
5. Usaremos el algoritmo de optimización de Adam.

In [22]:
# define the LSTM model
??? 


<a id="section2.6"></a>
# <font color="#004D7F" size=5>2.6. Crear puntos de control</font>

Usaremos el mejor conjunto de pesos (menor pérdida) para instanciar nuestro modelo generativo en la siguiente sección.

In [23]:
# define the checkpoint
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]


<a id="section2.7"></a>
# <font color="#004D7F" size=5>2.7. Resultados</font>

Utilizamos un número modesto de 20 épocas y un gran tamaño de batch de 128 patrones.

In [ ]:
# fit the model
??? 


Epoch 1/20
1283/1283 [==============================] - ETA: 0s - loss: 3.0190
Epoch 00001: loss improved from inf to 3.01901, saving model to weights-improvement-01-3.0190.hdf5
1283/1283 [==============================] - 301s 235ms/step - loss: 3.0190
Epoch 2/20
1283/1283 [==============================] - ETA: 0s - loss: 2.8445
Epoch 00002: loss improved from 3.01901 to 2.84446, saving model to weights-improvement-02-2.8445.hdf5
1283/1283 [==============================] - 297s 231ms/step - loss: 2.8445
Epoch 3/20
1283/1283 [==============================] - ETA: 0s - loss: 2.7635
Epoch 00003: loss improved from 2.84446 to 2.76349, saving model to weights-improvement-03-2.7635.hdf5
1283/1283 [==============================] - 299s 233ms/step - loss: 2.7635
Epoch 4/20
1033/1283 [=======================>......] - ETA: 56s - loss: 2.7060

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section3"></a>
# <font color="#004D7F" size=6>3. Generación de texto con LSTM</font>

La generación de texto utilizando la red LSTM entrenada es relativamente sencilla. 

<a id="section3.1"></a>
# <font color="#004D7F" size=5>3.1. Cargar los pesos de LSTM</font>

En primer lugar, cargamos los datos y definimos la red exactamente de la misma manera

Los pesos de la red se cargan desde un archivo de punto de control y no es necesario entrenar la red. 

In [ ]:
# load the network weights
filename = "weights-improvement-19-1.9435.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')


<a id="section3.2"></a>
# <font color="#004D7F" size=5>3.2. Convertir de entero a carácter</font>

Creamos un mapeo inverso que podamos usar para convertir los números enteros de nuevo en caracteres.

In [ ]:
# summarize the loaded data
int_to_char = dict((i, c) for i, c in enumerate(chars))

<a id="section3.3"></a>
# <font color="#004D7F" size=5>3.3. Resultados y evaluación</font>

Finalmente, necesitamos realmente hacer predicciones de manera que:
1. Comenzamos primero con una semilla como entrada
2. Generamos el siguiente carácter y 
3. Actualizamos la semilla para agregar el carácter generado al final y recortar el primer carácter. 

Este proceso se repite mientras queramos predecir nuevos caracteres (por ejemplo, una secuencia de 1000 caracteres de longitud). 

In [ ]:
# pick a random seed
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.") 


Podemos notar algunas observaciones sobre el texto generado.
* Generalmente se ajusta al formato de línea observado en el texto original de menos de 80 caracteres.
* Los caracteres están separados en grupos parecidos a palabras y la mayoría de los grupos son palabras reales en inglés (por ejemplo, _the, little_ y _was),_ pero muchos no (por ejemplo, _lott, tiie_ y _taede)._
* Algunas de las palabras en secuencia tienen sentido (por ejemplo, y el _white rabbit),_ pero otras muchas no (por ejemplo, _wese tilel)._

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section4"></a>
# <font color="#004D7F" size=6>4. LSTM más profunda</font>

Ahora, podemos intentar mejorar la calidad del texto generado creando una red mucho más profunda.

1. Definimos una única capa LSTM oculta con 256 unidades de memoria. 
2. La red utiliza un Dropout del 20%. 
3. Una capa LSTM de 256 unidades.
4. Otra capa Dropout del 20%. 
4. La capa de salida es una capa densa que utiliza la función de activación Softmax. 
4. Compilación con pérdida logarítmica (`categorical_crossentropy`)
5. Usaremos el algoritmo de optimización de Adam.

In [ ]:
# define the LSTM model
??? 


También cambiaremos el nombre de archivo de los pesos con puntos de control para que podamos distinguir entre los pesos de esta red y la anterior (agregando la palabra más grande en el nombre del archivo).

In [ ]:
# define the checkpoint
??? 


Finalmente, aumentaremos el número de épocas de entrenamiento de 20 a 50 y disminuiremos el tamaño del lote de 128 a 64 para darle a la red más oportunidades de actualizarse y aprender. 

In [ ]:
# fit the model
??? 


Como en la sección anterior, podemos usar este mejor modelo de la ejecución para generar texto. 

El único cambio que debemos realizar es la especificación de la topología de la red y desde qué archivo se van a generar los pesos de la red. 

In [ ]:
# load the network weights
filename = "???"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')
# pick a random seed
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section5"></a>
# <font color="#004D7F" size=6>5. Mejorar nuestro modelo</font>

A continuación, se muestra una muestra de ideas que tal vez desee investigar para mejorar aún más el modelo:
* Predecir menos de 1000 caracteres como salida para una semilla determinada.
* Eliminar toda la puntuación del texto fuente y, por tanto, del vocabulario de los modelos.
* Pruebe un One-Hot Encoding para las secuencias de entrada.
* Entrene al modelo en oraciones rellenas en lugar de secuencias aleatorias de caracteres.
* Aumentar el número de épocas de entrenamiento a 100 o más.
* Agregue Dropout a la capa de entrada visible y considere ajustar el porcentaje de Dropout.
* Ajuste el tamaño de batch, pruebe con un tamaño de batch de 1 como línea de base (muy lenta) y tamaños más grandes a partir de ahí.
* Agregue más unidades de memoria a las capas y / o más capas.
* Cambie las capas de LSTM para que tengan estado para mantener el estado en todos los batch.

<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-coffee" aria-hidden="true" style="color:#004D7F"></i> </font></div>